# Extract regional dataset from HydroSHEDS

In this notebook the regional dataset is extracted from the HydroSHEDS dataset.
The countries combined are: India, Bangladesh, China Xizang, Butan en Nepal.

In [ ]:
import geopandas as gp
import pandas as pd
import time
from matplotlib import pyplot as plt
from descartes import PolygonPatch

In [ ]:
water = gp.read_file('as_riv_15s/as_riv_15s.shp')
#water = water.set_index('ARCID')

Create spatial index

In [ ]:
start = time.time()
water_index = water.sindex
end = time.time()
print(end - start)

## Combining regions

In [ ]:
ind = gp.read_file('adm regions/IND_adm/IND_adm0.shp')

In [ ]:
bgd = gp.read_file('adm regions/BGD_adm/BGD_adm0.shp')

In [ ]:
chn = gp.read_file('adm regions/CHN_adm/CHN_adm0.shp')
chnprov = gp.read_file('adm regions/CHN_adm/CHN_adm1.shp')
#chnprov = chnprov[chnprov['NAME_1']=='Xizang' ]

In [ ]:
btn = gp.read_file('adm regions/BTN_adm/BTN_adm0.shp')

In [ ]:
npl = gp.read_file('adm regions/NPL_adm/NPL_adm0.shp')

In [ ]:
chn

In [ ]:
chn.loc[[0],'geometry'] = chnprov.loc[[28],'geometry'].values

In [ ]:
regions = pd.concat([ind,bgd,chn,btn,npl])

In [ ]:
regions.to_file('adm regions/combined.shp')

In [ ]:
regions_area = regions.convex_hull

In [ ]:
regions.boundary

Create single shape:

In [ ]:
region = regions.unary_union

## Finding edges in square region

In [ ]:
possible_matches_index = list(water_index.intersection(region.bounds))
possible_matches = water.iloc[possible_matches_index]
#precise_matches = possible_matches[possible_matches.intersects(region)]

In [ ]:
possible_matches.to_file("out/water_in_region_box.shp")

## Precise finding edges

First step is to divide the shape into squares.

In [ ]:
west, south, east, north = region.bounds

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
for polygon in region:
    patch = PolygonPatch(polygon, fc='#cccccc', ec='k', alpha=0.5, zorder=2)
    ax.add_patch(patch)

ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()

In [ ]:
import osmnx as ox

In [ ]:
geometry_cut = ox.quadrat_cut_geometry(region, quadrat_width=1)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
for polygon in geometry_cut:
    patch = PolygonPatch(polygon, fc='#cccccc', ec='k', alpha=0.5, zorder=2)
    ax.add_patch(patch)
    
ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()

To speed up calculation, first the matches for each square cut are found.
Then for each of those matches it is determined if the are in the shapeform.

In [ ]:
# find the points that intersect with each subpolygon and add them to points_within_geometry
sindex = water_index
points_within_geometry = pd.DataFrame()
i=0
for poly in geometry_cut:
    # buffer by the <1 micron dist to account for any space lost in the quadrat cutting
    # otherwise may miss point(s) that lay directly on quadrat line
    poly = poly.buffer(1e-14).buffer(0)

    # find approximate matches with r-tree, then precise matches from those approximate ones
    possible_matches_index = list(sindex.intersection(poly.bounds))
    possible_matches = water.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(poly)]
    points_within_geometry = points_within_geometry.append(precise_matches)
    
    i=i+1
    print("{:.0%}".format(i/len(geometry_cut)),end="\r")

In [ ]:
points_within_geometry.head()

In [ ]:
points_within_geometry.shape

In [ ]:
# drop duplicate points, if buffered poly caused an overlap on point(s) that lay directly on a quadrat line
points_within_geometry = points_within_geometry.drop_duplicates(subset=['ARCID'])
points_outside_geometry = water[~water.isin(points_within_geometry)]

In [ ]:
points_within_geometry.shape

Save the lines within the geometry

In [ ]:
points_within_geometry.to_file("out/water_in_region.shp")

Plot lines within or outside the geometry

In [ ]:
fig, ax = plt.subplots(1, figsize=(18,18))
for polygon in region:
    patch = PolygonPatch(polygon, fc='#cccccc', ec='k', alpha=0.5, zorder=2)
    ax.add_patch(patch)

#points_within_geometry.plot(ax=ax,color='b')
points_outside_geometry.plot(ax=ax,color='r')

ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()